In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
style.use('fivethirtyeight')
sns.set(style='darkgrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical

# specifically for models
from keras.layers import Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D, GlobalAveragePooling2D
from keras.layers import GlobalAvgPool2D as GAP, Dense, Dropout
from keras.applications.vgg19 import VGG19
from keras.applications import ResNet50V2

#callbacks
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  

# Task 1

## Data Ingestion

- We loop through all the images in all the directories, load each image using OpenCV, resize it into a fixed size (200) and append the image data and its corresponding label to the global lists IMAGE and LABEL, respectively. 
- Then, we return these lists to create the training and validation dataset for the models.

In [3]:
IMAGE=[]
LABEL=[]
IMG_SIZE=200

def assign_label(img,flower_category):
    return flower_category

def make_train_data(flower_category,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_category)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        IMAGE.append(np.array(img))
        LABEL.append(str(label))

FLOWER_DIR = []
flower_categories = ["Babi", "Calimerio", "Chrysanthemum", "Hydrangeas", "Lisianthus", "Pingpong", "Rosy", "Tana"]

for category in flower_categories:
    flower_dir = f"data/Flowers/{category}"
    FLOWER_DIR.append(flower_dir)
    make_train_data(category, flower_dir)
print('Total Image: ' + str(len(IMAGE)))

100%|███████████████████████████████████████████████████████████████████████████████| 623/623 [00:02<00:00, 271.30it/s]

Total Image: 4621


- We encode the labels using LabelEncoder, convert them to one-hot encoded vectors and store them in Y. 
- Then, we convert the image data to numpy array, scale the data to values between 0-1 and store them in X.

In [4]:
num_categories = 8

label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(LABEL)
Y=to_categorical(Y,num_categories)
X=np.array(IMAGE)
X=X/255

## Data Splitting

- We split the data into two set:
    + Training dataset: 80% of the total dataset.
    + Validation dataset: 20% of the total dataset.

In [5]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

# Confirm the shapes of the data splits
print("Train set shapes:", x_train.shape, y_train.shape)
print("Test set shapes:", x_test.shape, y_test.shape)

Train set shapes: (3696, 200, 200, 3) (3696, 8)
Test set shapes: (925, 200, 200, 3) (925, 8)


- We ensure that the same train-test split is generated every time we run the code, which is useful for reproducibility.
- This allows you to compare the results of different models or hyperparameters on the same train-test split.

In [6]:
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

## Data Augmentation

- By randomly applying changes to the original images, these configurations enable the development of augmented images, which are slightly changed copies of the original images. 
- This can assist to increase the size and variability of the training set, which can improve the model's performance.

In [7]:
data_aug = ImageDataGenerator(
                featurewise_center=False,  # set input mean to 0 over the dataset
                samplewise_center=False,  # set each sample mean to 0
                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                samplewise_std_normalization=False,  # divide each input by its std
                zca_whitening=False,  # apply ZCA whitening
                rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                zoom_range = 0.1, # Randomly zoom image 
                width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
                height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                horizontal_flip=True,  # randomly flip images
                vertical_flip=False)  # randomly flip images

## Models

### CNN Model

#### Build and Train Model

- We design this model with 5 convolution layers with max pooling layers in between, followed by a flatten layer, a dense layer and an output layer.

In [8]:
cnn_model = Sequential()
cnn_model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (IMG_SIZE,IMG_SIZE,3)))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))


cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

cnn_model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

cnn_model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same',activation ='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))


cnn_model.add(Flatten())
cnn_model.add(Dense(512))
cnn_model.add(Activation('relu'))
cnn_model.add(Dense(num_categories, activation = "softmax"))

- We set up the optimizer, loss function and evaluation metric before training the model.
- Then, we print a summary of the model architecture which includes the number of parameters in each layer and the output shape of each layer as well.

In [9]:
cnn_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 100, 100, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 50, 50, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 96)        55392     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 25, 25, 96)       0

- The batch size and total number of epochs are assigned to the model. 
-
- Then, we fit the training data into the model.

In [10]:
batch_size = 64
epochs = 70

callbacks = [EarlyStopping(patience=5, restore_best_weights=True), ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)]

cnn_history = cnn_model.fit(data_aug.flow(x_train,y_train, batch_size= batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

Epoch 1/70
57/57 [==============================] - 290s 5s/step - loss: 1.9816 - accuracy: 0.2037 - val_loss: 1.9294 - val_accuracy: 0.2454
Epoch 2/70
57/57 [==============================] - ETA: 0s - loss: 1.9074 - accuracy: 0.2481

KeyboardInterrupt: 

#### Loss Function

In [ ]:
plt.plot(cnn_history.history['loss'])
plt.plot(cnn_history.history['val_loss'])
plt.title('CNN Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

#### Accuracy

In [ ]:
plt.plot(cnn_history.history['accuracy'])
plt.plot(cnn_history.history['val_accuracy'])
plt.title('CNN Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

### ResNet50V2 Model

In [ ]:
ResNet50V2_base_model = ResNet50V2(input_shape=(IMG_SIZE,IMG_SIZE,3), include_top=False)

for layer in ResNet50V2_base_model.layers[:170]:
    layer.trainable = False

ResNet50V2_model = Sequential([
    ResNet50V2_base_model,
    GAP(),
    Dense(512, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.4),
    Dense(num_categories, activation='softmax')
])

In [ ]:
ResNet50V2_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy' , metrics = ['accuracy'])

ResNet50V2_model.summary()

In [ ]:
ResNet50V2_history = ResNet50V2_model.fit(data_aug.flow(x_train,y_train, batch_size= batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size, callbacks = callbacks)

In [ ]:
plt.plot(ResNet50V2_history.history['loss'])
plt.plot(ResNet50V2_history.history['val_loss'])
plt.title('ResNet50V2 Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(ResNet50V2_history.history['accuracy'])
plt.plot(ResNet50V2_history.history['val_accuracy'])
plt.title('ResNet50V2 Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

### VGG19 Model

#### Build and Train Model

- This model is built on top of the pre-trained VGG19 model from Keras.
- We firstly freeze the weights of the first 19 layers of the pre-trained VGG19 to simplify the model.
- Then, we create our model by adding more layers to the model: 
    + A MaxPool2D layer to downsample the feature maps.
    + A Flatten layer to convert output into a 1D array.
    + A Dense layer with a softmax activation function to classify the images into one of the eight categories. 

In [ ]:
VGG19_based_model = VGG19(input_shape=(IMG_SIZE,IMG_SIZE,3), include_top=False)

for layer in VGG19_based_model.layers[:19]:
    layer.trainable = False

VGG19_model = Sequential([
    VGG19_based_model,
    MaxPool2D((2,2) , strides = 2),
    Flatten(),
    Dense(256, activation='relu', kernel_initializer='he_normal'),
    Dense(num_categories , activation='softmax')
])

- We set up the optimizer, loss function and evaluation metric before training the model.
- Additionally, we set up a callback function to reduce the learning rate when the validation accuracy does not improve after 2 epochs. This helps the model to converge better and avoid overfitting.
- Then, we print a summary of the model architecture which includes the number of parameters in each layer and the output shape of each layer as well.


In [ ]:
VGG19_model.compile(optimizer =Adam(lr=0.001), loss = 'categorical_crossentropy' , metrics = ['accuracy'])

VGG19_model.summary()

- The batch size and total number of epochs are assigned to the model. 
- Then, we fit the training data into the model.

In [ ]:
VGG19_history = VGG19_model.fit(data_aug.flow(x_train,y_train, batch_size= batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size, callbacks = callbacks)

#### Loss Function

In [ ]:
plt.plot(VGG19_history.history['loss'])
plt.plot(VGG19_history.history['val_loss'])
plt.title('vgg19 Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

#### Accuracy

In [ ]:
plt.plot(VGG19_history.history['accuracy'])
plt.plot(VGG19_history.history['val_accuracy'])
plt.title('VGG19 Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

## Predictions on Validation Set

In [ ]:
pred=VGG19_model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

### Correct Classified Images

In [ ]:
i=0
cor_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        cor_class.append(i)
    if(len(cor_class)==9):
        break

warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(3,3)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[cor_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(label_encoder.inverse_transform([pred_digits[cor_class[count]]]))+"\n"+"Actual Flower : "+str(label_encoder.inverse_transform(np.argmax(y_test[cor_class[count]],axis=0))))
        plt.tight_layout()
        count+=1

### Missclassified Images

In [ ]:
i=0
mis_class=[]

for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==9):
        break

warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(3,3)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[mis_class[count]])
        ax[i,j].set_title("Predicted Flower : "+str(label_encoder.inverse_transform([pred_digits[mis_class[count]]]))+"\n"+"Actual Flower : "+str(label_encoder.inverse_transform(np.argmax(y_test[mis_class[count]]))))
        plt.tight_layout()
        count+=1